# Money Modeling Software v1.0 - Complete Example with Firm Agents

This notebook implements the concrete example from pages 7-10 of the Money modeling software.pdf document
using the new Firm agent type for proper business entity representation.

## Example Timeline

- **t0**: Initial setup with 4 firm agents
  
- **t1**: Agent 3 delivers 10 machines to Agent 2
- **t2**: Agent 4 pays $110 to Agent 2; Agent 2 delivers 10 machines to Agent 4
  
- **t3**: Agent 2 pays $110 to Agent 1 (bond settlement)

## Expected Final State
- Agent 1: $110 cash
- Agent 2: Nothing
- Agent 3: Nothing
- Agent 4: 10 machines

In [1]:
# Import required modules
from bilancio.engines.system import System
from bilancio.engines.simulation import run_day, run_until_stable
from bilancio.domain.agents.firm import Firm
from bilancio.domain.agents.central_bank import CentralBank
from bilancio.domain.instruments.credit import Payable
from bilancio.domain.instruments.delivery import DeliveryObligation
from bilancio.analysis.visualization import (
    display_agent_balance_table, 
    display_multiple_agent_balances,
    display_events,
    display_events_for_day
)
from bilancio.analysis.balances import agent_balance
from decimal import Decimal

# Initialize the system
sys = System()
print(f"System initialized at day {sys.state.day}")

System initialized at day 0


In [2]:
# Create agents during system setup phase
with sys.setup():
    # Create Central Bank (required for cash issuance)
    cb = CentralBank(id="CB", name="Central Bank", kind="central_bank")
    sys.add_agent(cb)

    # Create the 4 agents as Firms (business entities)
    agent1 = Firm(id="Agent1", name="Firm 1", kind="firm")
    agent2 = Firm(id="Agent2", name="Firm 2", kind="firm") 
    agent3 = Firm(id="Agent3", name="Manufacturing Firm 3", kind="firm")  # Has machines
    agent4 = Firm(id="Agent4", name="Trading Firm 4", kind="firm")  # Has cash

    sys.add_agent(agent1)
    sys.add_agent(agent2)
    sys.add_agent(agent3)
    sys.add_agent(agent4)

print("✓ All agents created as Firms")
for agent_id in ["Agent1", "Agent2", "Agent3", "Agent4"]:
    print(f"  - {agent_id}: {sys.state.agents[agent_id].name} (kind={sys.state.agents[agent_id].kind})")

✓ All agents created as Firms
  - Agent1: Firm 1 (kind=firm)
  - Agent2: Firm 2 (kind=firm)
  - Agent3: Manufacturing Firm 3 (kind=firm)
  - Agent4: Trading Firm 4 (kind=firm)


## Set up Initial Balance Sheet Positions (t0)

In [3]:
# Set up initial positions during system setup phase
with sys.setup():
    # 1. Bond: Agent 2 owes Agent 1 $110 (due day 3)
    bond = Payable(
        id=sys.new_contract_id("BOND"),
        kind="payable",
        amount=110,
        denom="USD",
        asset_holder_id="Agent1",
        liability_issuer_id="Agent2",
        due_day=3
    )
    sys.add_contract(bond)
    print("✓ Bond created: Agent2 owes Agent1 $110 (due day 3)")

    # 2. Agent 3's machine inventory (manufacturing firm with production assets)
    machines_inv_id = sys.create_stock(
        owner_id="Agent3",
        sku="machines",
        quantity=10,
        unit_price=Decimal("0"),
        divisible=True
    )
    print("✓ Inventory: Agent3 (manufacturer) owns 10 machines")

    # 3. Delivery obligation: Agent 3 owes Agent 2 10 machines (due day 1)
    delivery1 = DeliveryObligation(
        id=sys.new_contract_id("DEL1"),
        kind="delivery_obligation",
        amount=10,  # quantity promised
        sku="machines",
        unit_price=Decimal("0"),
        denom="USD",
        asset_holder_id="Agent2",
        liability_issuer_id="Agent3",
        due_day=1
    )
    sys.add_contract(delivery1)
    print("✓ Delivery contract: Agent3 must deliver 10 machines to Agent2 (due day 1)")

    # 4. Agent 4's cash (trading firm with liquidity)
    sys.mint_cash("Agent4", 110)
    print("✓ Cash minted: Agent4 (trader) has $110")

    # 5. Payable: Agent 4 owes Agent 2 $110 (due day 2)
    payment = Payable(
        id=sys.new_contract_id("PAY"),
        kind="payable",
        amount=110,
        denom="USD",
        asset_holder_id="Agent2",
        liability_issuer_id="Agent4",
        due_day=2
    )
    sys.add_contract(payment)
    print("✓ Payment obligation: Agent4 owes Agent2 $110 (due day 2)")

    # 6. Delivery obligation: Agent 2 owes Agent 4 10 machines (due day 2)
    delivery2 = DeliveryObligation(
        id=sys.new_contract_id("DEL2"),
        kind="delivery_obligation",
        amount=10,  # quantity promised
        sku="machines",
        unit_price=Decimal("0"),
        denom="USD",
        asset_holder_id="Agent4",
        liability_issuer_id="Agent2",
        due_day=2
    )
    sys.add_contract(delivery2)
    print("✓ Delivery contract: Agent2 must deliver 10 machines to Agent4 (due day 2)")

print("\n📊 All initial positions established")

✓ Bond created: Agent2 owes Agent1 $110 (due day 3)
✓ Inventory: Agent3 (manufacturer) owns 10 machines
✓ Delivery contract: Agent3 must deliver 10 machines to Agent2 (due day 1)
✓ Cash minted: Agent4 (trader) has $110
✓ Payment obligation: Agent4 owes Agent2 $110 (due day 2)
✓ Delivery contract: Agent2 must deliver 10 machines to Agent4 (due day 2)

📊 All initial positions established


## Display Initial State (t0)

In [4]:
print("="*70)
print("INITIAL BALANCE SHEETS (t0 = Day 0)")
print("="*70)

# Use the built-in display function for all agents
display_multiple_agent_balances(
    system=sys,
    items=["Agent1", "Agent2", "Agent3", "Agent4"],
    format='rich'
)

INITIAL BALANCE SHEETS (t0 = Day 0)


              Firm 1                                 Firm 2                          Manufacturing Firm 3          
              (firm)                                 (firm)                                 (firm)                 
╭────────────────────┬────────────╮    ╭────────────────────┬────────────╮    ╭────────────────────┬────────────╮  
│ Item               │     Amount │    │ Item               │     Amount │    │ Item               │     Amount │  
├────────────────────┼────────────┤    ├────────────────────┼────────────┤    ├────────────────────┼────────────┤  
│ ASSETS             │            │    │ ASSETS             │            │    │ ASSETS             │            │  
│ payable            │        110 │    │ machines recv [10] │          0 │    │ machines [10]      │          0 │  
│                    │            │    │ payable            │        110 │    │                    │            │  
├────────────────────┼────────────┤    │                    │            │    ├────────────────────┼────────────┤  
│ LIABILITIES        │            │    ├────────────────────┼────────────┤    │ LIABILITIES        │            │  
│                    │            │    │ LIABILITIES        │            │    │ machines oblig     │          0 │  
├────────────────────┼────────────┤    │ machines oblig     │          0 │    │ [10]               │            │  
│ Total Financial    │        110 │    │ [10]               │            │    │                    │            │  
│ Total Liab.        │          0 │    │ payable            │        110 │    ├────────────────────┼────────────┤  
│ Net Financial      │       +110 │    │                    │            │    │ Total Financial    │          0 │  
╰────────────────────┴────────────╯    ├────────────────────┼────────────┤    │ Total Liab.        │          0 │  
                                       │ Total Financial    │        110 │    │ Net Financial      │          0 │  
                                       │ Total Liab.        │        110 │    ╰────────────────────┴────────────╯  
                                       │ Net Financial      │          0 │                                         
                                       ╰────────────────────┴────────────╯                                         
          Trading Firm 4                                                                                           
              (firm)                                                                                               
╭────────────────────┬────────────╮                                                                                
│ Item               │     Amount │                                                                                
├────────────────────┼────────────┤                                                                                
│ ASSETS             │            │                                                                                
│ machines recv [10] │          0 │                                                                                
│ cash               │        110 │                                                                                
│                    │            │                                                                                
├────────────────────┼────────────┤                                                                                
│ LIABILITIES        │            │                                                                                
│ payable            │        110 │                                                                                
│                    │            │                                                                                
├────────────────────┼────────────┤                                                                                
│ Total Financial    │        110 │                     

## Run Simulation Day by Day

In [5]:
# Run to Day 1 and settle Day 1 obligations
print("\n" + "="*70)
print("ADVANCE TO DAY 1 - Machine Delivery")
print("="*70)

print(f"\n📍 Starting at day {sys.state.day}")
print("\n📋 Obligations due on day 1:")
print("  • Agent3 must deliver 10 machines to Agent2")

# First run_day advances from day 0 to day 1
run_day(sys)
print(f"\n✅ Advanced to day {sys.state.day}")

# Day 0 had no obligations, just setup events
print("\n🔍 Day 0 events (setup and phases):")
display_events_for_day(sys, 0)

# Now we're on day 1, run_day again to settle day 1 obligations and advance to day 2
print("\n" + "-"*50)
print("📍 Now on day 1, settling day 1 obligations...")
run_day(sys)

print(f"\n✅ Advanced to day {sys.state.day}")
print("\n🔍 Day 1 events (machine delivery):")
display_events_for_day(sys, 1)

print("\n📊 Result after Day 1 settlements:")
print("  • Agent2: Now has 10 machines (received from Agent3)")
print("  • Agent3: No longer has machines (delivered to Agent2)")

# Show balance sheets
print("\nBalance sheets after day 1 settlements:")
display_multiple_agent_balances(
    system=sys,
    items=["Agent1", "Agent2", "Agent3", "Agent4"],
    format='rich'
)


ADVANCE TO DAY 1 - Machine Delivery

📍 Starting at day 0

📋 Obligations due on day 1:
  • Agent3 must deliver 10 machines to Agent2

✅ Advanced to day 1

🔍 Day 0 events (setup and phases):
  🏭 Stock created: Agent3 gets 10 machines
  💰 Cash minted: $110 to Agent4
  ⏰ Settlement phase begins (checking due obligations)
  ⏳ End of day phase
  • PhaseC: {'kind': 'PhaseC', 'day': 0, 'phase': 'simulation'}

--------------------------------------------------
📍 Now on day 1, settling day 1 obligations...

✅ Advanced to day 2

🔍 Day 1 events (machine delivery):
  ⏰ Settlement phase begins (checking due obligations)
  ⏳ End of day phase
  📦 Stock transferred: Agent3 → Agent2: 10 machines
    └─ Delivery obligation removed from books
  ✅ Delivery settled: Agent3 → Agent2: 10 machines
  • PhaseC: {'kind': 'PhaseC', 'day': 1, 'phase': 'simulation'}

📊 Result after Day 1 settlements:
  • Agent2: Now has 10 machines (received from Agent3)
  • Agent3: No longer has machines (delivered to Agent2)

Bal

              Firm 1                                 Firm 2                          Manufacturing Firm 3          
              (firm)                                 (firm)                                 (firm)                 
╭────────────────────┬────────────╮    ╭────────────────────┬────────────╮    ╭────────────────────┬────────────╮  
│ Item               │     Amount │    │ Item               │     Amount │    │ Item               │     Amount │  
├────────────────────┼────────────┤    ├────────────────────┼────────────┤    ├────────────────────┼────────────┤  
│ ASSETS             │            │    │ ASSETS             │            │    │ ASSETS             │            │  
│ payable            │        110 │    │ machines [10]      │          0 │    │                    │            │  
│                    │            │    │ payable            │        110 │    ├────────────────────┼────────────┤  
├────────────────────┼────────────┤    │                    │            │    │ LIABILITIES        │            │  
│ LIABILITIES        │            │    ├────────────────────┼────────────┤    │                    │            │  
│                    │            │    │ LIABILITIES        │            │    ├────────────────────┼────────────┤  
├────────────────────┼────────────┤    │ machines oblig     │          0 │    │ Total Financial    │          0 │  
│ Total Financial    │        110 │    │ [10]               │            │    │ Total Liab.        │          0 │  
│ Total Liab.        │          0 │    │ payable            │        110 │    │ Net Financial      │          0 │  
│ Net Financial      │       +110 │    │                    │            │    ╰────────────────────┴────────────╯  
╰────────────────────┴────────────╯    ├────────────────────┼────────────┤                                         
                                       │ Total Financial    │        110 │                                         
                                       │ Total Liab.        │        110 │                                         
                                       │ Net Financial      │          0 │                                         
                                       ╰────────────────────┴────────────╯                                         
          Trading Firm 4                                                                                           
              (firm)                                                                                               
╭────────────────────┬────────────╮                                                                                
│ Item               │     Amount │                                                                                
├────────────────────┼────────────┤                                                                                
│ ASSETS             │            │                                                                                
│ machines recv [10] │          0 │                                                                                
│ cash               │        110 │                                                                                
│                    │            │                                                                                
├────────────────────┼────────────┤                                                                                
│ LIABILITIES        │            │                                                                                
│ payable            │        110 │                                                                                
│                    │            │                                                                                
├────────────────────┼────────────┤                                                                                
│ Total Financial    │        110 │                     

In [6]:
# Run to Day 2 and settle Day 2 obligations
print("\n" + "="*70)
print("ADVANCE TO DAY 2 - Payment and Delivery Exchange")
print("="*70)

print(f"\n📍 Currently at day {sys.state.day}")
print("\n📋 Obligations due on day 2:")
print("  • Agent4 must pay $110 to Agent2")
print("  • Agent2 must deliver 10 machines to Agent4")

# We're on day 2, run_day to settle day 2 obligations and advance to day 3
run_day(sys)

print(f"\n✅ Advanced to day {sys.state.day}")
print("\n🔍 Day 2 events (payment and delivery):")
display_events_for_day(sys, 2)

print("\n📊 Result after Day 2 settlements:")
print("  • Agent2: Has $110 cash (received from Agent4)")
print("  • Agent4: Has 10 machines (received from Agent2)")
print("  • Agent4: No longer has cash (paid to Agent2)")

# Show balance sheets
print("\nBalance sheets after day 2 settlements:")
display_multiple_agent_balances(
    system=sys,
    items=["Agent1", "Agent2", "Agent3", "Agent4"],
    format='rich'
)


ADVANCE TO DAY 2 - Payment and Delivery Exchange

📍 Currently at day 2

📋 Obligations due on day 2:
  • Agent4 must pay $110 to Agent2
  • Agent2 must deliver 10 machines to Agent4

✅ Advanced to day 3

🔍 Day 2 events (payment and delivery):
  ⏰ Settlement phase begins (checking due obligations)
  ⏳ End of day phase
  💵 Cash transferred: Agent4 → Agent2: $110
  ✅ Payment settled: Agent4 → Agent2: $110
  📦 Stock transferred: Agent2 → Agent4: 10 machines
    └─ Delivery obligation removed from books
  ✅ Delivery settled: Agent2 → Agent4: 10 machines
  • PhaseC: {'kind': 'PhaseC', 'day': 2, 'phase': 'simulation'}

📊 Result after Day 2 settlements:
  • Agent2: Has $110 cash (received from Agent4)
  • Agent4: Has 10 machines (received from Agent2)
  • Agent4: No longer has cash (paid to Agent2)

Balance sheets after day 2 settlements:


              Firm 1                                 Firm 2                          Manufacturing Firm 3          
              (firm)                                 (firm)                                 (firm)                 
╭────────────────────┬────────────╮    ╭────────────────────┬────────────╮    ╭────────────────────┬────────────╮  
│ Item               │     Amount │    │ Item               │     Amount │    │ Item               │     Amount │  
├────────────────────┼────────────┤    ├────────────────────┼────────────┤    ├────────────────────┼────────────┤  
│ ASSETS             │            │    │ ASSETS             │            │    │ ASSETS             │            │  
│ payable            │        110 │    │ cash               │        110 │    │                    │            │  
│                    │            │    │                    │            │    ├────────────────────┼────────────┤  
├────────────────────┼────────────┤    ├────────────────────┼────────────┤    │ LIABILITIES        │            │  
│ LIABILITIES        │            │    │ LIABILITIES        │            │    │                    │            │  
│                    │            │    │ payable            │        110 │    ├────────────────────┼────────────┤  
├────────────────────┼────────────┤    │                    │            │    │ Total Financial    │          0 │  
│ Total Financial    │        110 │    ├────────────────────┼────────────┤    │ Total Liab.        │          0 │  
│ Total Liab.        │          0 │    │ Total Financial    │        110 │    │ Net Financial      │          0 │  
│ Net Financial      │       +110 │    │ Total Liab.        │        110 │    ╰────────────────────┴────────────╯  
╰────────────────────┴────────────╯    │ Net Financial      │          0 │                                         
                                       ╰────────────────────┴────────────╯                                         
          Trading Firm 4                                                                                           
              (firm)                                                                                               
╭────────────────────┬────────────╮                                                                                
│ Item               │     Amount │                                                                                
├────────────────────┼────────────┤                                                                                
│ ASSETS             │            │                                                                                
│ machines [10]      │          0 │                                                                                
│                    │            │                                                                                
├────────────────────┼────────────┤                                                                                
│ LIABILITIES        │            │                                                                                
│                    │            │                                                                                
├────────────────────┼────────────┤                                                                                
│ Total Financial    │          0 │                                                                                
│ Total Liab.        │          0 │                                                                                
│ Net Financial      │          0 │                                                                                
╰────────────────────┴────────────╯

In [7]:
# Run to Day 3 and settle Day 3 obligations
print("\n" + "="*70)
print("ADVANCE TO DAY 3 - Bond Settlement")
print("="*70)

print(f"\n📍 Currently at day {sys.state.day}")
print("\n📋 Obligations due on day 3:")
print("  • Agent2 must pay $110 to Agent1 (bond maturity)")

# We're on day 3, run_day to settle day 3 obligations and advance to day 4
run_day(sys)

print(f"\n✅ Advanced to day {sys.state.day}")
print("\n🔍 Day 3 events (bond settlement):")
display_events_for_day(sys, 3)

print("\n📊 Result after Day 3 settlements:")
print("  • Agent1: Has $110 cash (received from Agent2)")
print("  • Agent2: Has nothing (paid cash to Agent1)")

# Show balance sheets
print("\nBalance sheets after day 3 settlements:")
display_multiple_agent_balances(
    system=sys,
    items=["Agent1", "Agent2", "Agent3", "Agent4"],
    format='rich'
)


ADVANCE TO DAY 3 - Bond Settlement

📍 Currently at day 3

📋 Obligations due on day 3:
  • Agent2 must pay $110 to Agent1 (bond maturity)

✅ Advanced to day 4

🔍 Day 3 events (bond settlement):
  ⏰ Settlement phase begins (checking due obligations)
  ⏳ End of day phase
  💵 Cash transferred: Agent2 → Agent1: $110
  ✅ Payment settled: Agent2 → Agent1: $110
  • PhaseC: {'kind': 'PhaseC', 'day': 3, 'phase': 'simulation'}

📊 Result after Day 3 settlements:
  • Agent1: Has $110 cash (received from Agent2)
  • Agent2: Has nothing (paid cash to Agent1)

Balance sheets after day 3 settlements:


              Firm 1                                 Firm 2                          Manufacturing Firm 3          
              (firm)                                 (firm)                                 (firm)                 
╭────────────────────┬────────────╮    ╭────────────────────┬────────────╮    ╭────────────────────┬────────────╮  
│ Item               │     Amount │    │ Item               │     Amount │    │ Item               │     Amount │  
├────────────────────┼────────────┤    ├────────────────────┼────────────┤    ├────────────────────┼────────────┤  
│ ASSETS             │            │    │ ASSETS             │            │    │ ASSETS             │            │  
│ cash               │        110 │    │                    │            │    │                    │            │  
│                    │            │    ├────────────────────┼────────────┤    ├────────────────────┼────────────┤  
├────────────────────┼────────────┤    │ LIABILITIES        │            │    │ LIABILITIES        │            │  
│ LIABILITIES        │            │    │                    │            │    │                    │            │  
│                    │            │    ├────────────────────┼────────────┤    ├────────────────────┼────────────┤  
├────────────────────┼────────────┤    │ Total Financial    │          0 │    │ Total Financial    │          0 │  
│ Total Financial    │        110 │    │ Total Liab.        │          0 │    │ Total Liab.        │          0 │  
│ Total Liab.        │          0 │    │ Net Financial      │          0 │    │ Net Financial      │          0 │  
│ Net Financial      │       +110 │    ╰────────────────────┴────────────╯    ╰────────────────────┴────────────╯  
╰────────────────────┴────────────╯                                                                                
          Trading Firm 4                                                                                           
              (firm)                                                                                               
╭────────────────────┬────────────╮                                                                                
│ Item               │     Amount │                                                                                
├────────────────────┼────────────┤                                                                                
│ ASSETS             │            │                                                                                
│ machines [10]      │          0 │                                                                                
│                    │            │                                                                                
├────────────────────┼────────────┤                                                                                
│ LIABILITIES        │            │                                                                                
│                    │            │                                                                                
├────────────────────┼────────────┤                                                                                
│ Total Financial    │          0 │                                                                                
│ Total Liab.        │          0 │                                                                                
│ Net Financial      │          0 │                                                                                
╰────────────────────┴────────────╯

## Final State Verification

In [8]:
print("\n" + "="*70)
print("FINAL BALANCE SHEETS (After t3)")
print("="*70)

# Display all agents' final balance sheets
display_multiple_agent_balances(
    system=sys,
    items=["Agent1", "Agent2", "Agent3", "Agent4"],
    format='rich'
)

print("\n" + "="*70)
print("VERIFICATION AGAINST EXPECTED")
print("="*70)

# Verify each agent using the agent_balance function
checks = []

# Agent 1 should have $110 cash
balance1 = agent_balance(sys, "Agent1")
a1_cash = balance1.assets_by_kind.get("cash", 0)
check1 = a1_cash == 110
checks.append(check1)
print(f"\nAgent1 has $110 cash: {'✅' if check1 else '❌'} (actual: ${a1_cash})")

# Agent 2 should have nothing
balance2 = agent_balance(sys, "Agent2")
check2 = (balance2.total_financial_assets == 0 and 
          balance2.total_financial_liabilities == 0 and
          balance2.total_inventory_value == 0)
checks.append(check2)
print(f"Agent2 has no assets/liabilities: {'✅' if check2 else '❌'}")

# Agent 3 should have nothing
balance3 = agent_balance(sys, "Agent3")
check3 = (balance3.total_financial_assets == 0 and 
          balance3.total_financial_liabilities == 0 and
          balance3.total_inventory_value == 0)
checks.append(check3)
print(f"Agent3 has no assets/liabilities: {'✅' if check3 else '❌'}")

# Agent 4 should have 10 machines
balance4 = agent_balance(sys, "Agent4")
a4_machines = balance4.inventory_by_sku.get("machines", {}).get("quantity", 0)
check4 = a4_machines == 10
checks.append(check4)
print(f"Agent4 has 10 machines: {'✅' if check4 else '❌'} (actual: {a4_machines})")

print("\n" + "="*70)
if all(checks):
    print("🎉 SUCCESS! The simulation perfectly matches the PDF example!")
    print("\nKey achievements:")
    print("✓ Firm agents can hold cash and settle payments")
    print("✓ Delivery obligations automatically settle on due dates")
    print("✓ Stock inventory properly transferred between agents")
    print("✓ All temporal obligations executed correctly")
    print("✓ Final state exactly matches specification")
    print("\n📊 The bilancio system successfully implements Version 1.0 requirements!")
else:
    print("⚠️  Some checks failed. Debugging needed.")
    print("\nCheck the event log for details:")
    
    # Show all settlement events
    print("\nAll settlements:")
    for i, event in enumerate(sys.state.events):
        if "Settled" in event.get("kind", ""):
            print(f"  {i}. Day {event.get('day', '?')}: {event}")


FINAL BALANCE SHEETS (After t3)


              Firm 1                                 Firm 2                          Manufacturing Firm 3          
              (firm)                                 (firm)                                 (firm)                 
╭────────────────────┬────────────╮    ╭────────────────────┬────────────╮    ╭────────────────────┬────────────╮  
│ Item               │     Amount │    │ Item               │     Amount │    │ Item               │     Amount │  
├────────────────────┼────────────┤    ├────────────────────┼────────────┤    ├────────────────────┼────────────┤  
│ ASSETS             │            │    │ ASSETS             │            │    │ ASSETS             │            │  
│ cash               │        110 │    │                    │            │    │                    │            │  
│                    │            │    ├────────────────────┼────────────┤    ├────────────────────┼────────────┤  
├────────────────────┼────────────┤    │ LIABILITIES        │            │    │ LIABILITIES        │            │  
│ LIABILITIES        │            │    │                    │            │    │                    │            │  
│                    │            │    ├────────────────────┼────────────┤    ├────────────────────┼────────────┤  
├────────────────────┼────────────┤    │ Total Financial    │          0 │    │ Total Financial    │          0 │  
│ Total Financial    │        110 │    │ Total Liab.        │          0 │    │ Total Liab.        │          0 │  
│ Total Liab.        │          0 │    │ Net Financial      │          0 │    │ Net Financial      │          0 │  
│ Net Financial      │       +110 │    ╰────────────────────┴────────────╯    ╰────────────────────┴────────────╯  
╰────────────────────┴────────────╯                                                                                
          Trading Firm 4                                                                                           
              (firm)                                                                                               
╭────────────────────┬────────────╮                                                                                
│ Item               │     Amount │                                                                                
├────────────────────┼────────────┤                                                                                
│ ASSETS             │            │                                                                                
│ machines [10]      │          0 │                                                                                
│                    │            │                                                                                
├────────────────────┼────────────┤                                                                                
│ LIABILITIES        │            │                                                                                
│                    │            │                                                                                
├────────────────────┼────────────┤                                                                                
│ Total Financial    │          0 │                                                                                
│ Total Liab.        │          0 │                                                                                
│ Net Financial      │          0 │                                                                                
╰────────────────────┴────────────╯


VERIFICATION AGAINST EXPECTED

Agent1 has $110 cash: ✅ (actual: $110)
Agent2 has no assets/liabilities: ✅
Agent3 has no assets/liabilities: ✅
Agent4 has 10 machines: ✅ (actual: 10)

🎉 SUCCESS! The simulation perfectly matches the PDF example!

Key achievements:
✓ Firm agents can hold cash and settle payments
✓ Delivery obligations automatically settle on due dates
✓ Stock inventory properly transferred between agents
✓ All temporal obligations executed correctly
✓ Final state exactly matches specification

📊 The bilancio system successfully implements Version 1.0 requirements!


In [9]:
# Display all events in chronological order
print("\n" + "="*70)
print("COMPLETE EVENT LOG")
print("="*70)

# Use the new display_events function from the visualization module
display_events(sys.state.events, format='detailed')


COMPLETE EVENT LOG

📅 Day 0:
  🏭 Stock created: Agent3 gets 10 machines
  💰 Cash minted: $110 to Agent4
  ⏰ Settlement phase begins (checking due obligations)
  ⏳ End of day phase
  • PhaseC: {'kind': 'PhaseC', 'day': 0, 'phase': 'simulation'}

📅 Day 1:
  ⏰ Settlement phase begins (checking due obligations)
  ⏳ End of day phase
  📦 Stock transferred: Agent3 → Agent2: 10 machines
    └─ Delivery obligation removed from books
  ✅ Delivery settled: Agent3 → Agent2: 10 machines
  • PhaseC: {'kind': 'PhaseC', 'day': 1, 'phase': 'simulation'}

📅 Day 2:
  ⏰ Settlement phase begins (checking due obligations)
  ⏳ End of day phase
  💵 Cash transferred: Agent4 → Agent2: $110
  ✅ Payment settled: Agent4 → Agent2: $110
  📦 Stock transferred: Agent2 → Agent4: 10 machines
    └─ Delivery obligation removed from books
  ✅ Delivery settled: Agent2 → Agent4: 10 machines
  • PhaseC: {'kind': 'PhaseC', 'day': 2, 'phase': 'simulation'}

📅 Day 3:
  ⏰ Settlement phase begins (checking due obligations)
  ⏳ E

In [10]:
print("\n" + "="*70)
print("ALTERNATIVE: RUN UNTIL STABLE DEMONSTRATION")
print("="*70)

# Create a fresh system to demonstrate run_until_stable
print("\n📌 Creating a fresh system to demonstrate automated simulation...")
fresh_sys = System()

with fresh_sys.setup():
    # Recreate the same initial setup
    cb = CentralBank(id="CB", name="Central Bank", kind="central_bank")
    fresh_sys.add_agent(cb)
    
    agent1 = Firm(id="Agent1", name="Firm 1", kind="firm")
    agent2 = Firm(id="Agent2", name="Firm 2", kind="firm")
    agent3 = Firm(id="Agent3", name="Manufacturing Firm 3", kind="firm")
    agent4 = Firm(id="Agent4", name="Trading Firm 4", kind="firm")
    
    fresh_sys.add_agent(agent1)
    fresh_sys.add_agent(agent2)
    fresh_sys.add_agent(agent3)
    fresh_sys.add_agent(agent4)
    
    # Set up the same initial conditions
    bond = Payable(
        id=fresh_sys.new_contract_id("BOND"),
        kind="payable",
        amount=110,
        denom="USD",
        asset_holder_id="Agent1",
        liability_issuer_id="Agent2",
        due_day=3
    )
    fresh_sys.add_contract(bond)
    
    fresh_sys.create_stock(
        owner_id="Agent3",
        sku="machines",
        quantity=10,
        unit_price=Decimal("0"),
        divisible=True
    )
    
    delivery1 = DeliveryObligation(
        id=fresh_sys.new_contract_id("DEL1"),
        kind="delivery_obligation",
        amount=10,
        sku="machines",
        unit_price=Decimal("0"),
        denom="USD",
        asset_holder_id="Agent2",
        liability_issuer_id="Agent3",
        due_day=1
    )
    fresh_sys.add_contract(delivery1)
    
    fresh_sys.mint_cash("Agent4", 110)
    
    payment = Payable(
        id=fresh_sys.new_contract_id("PAY"),
        kind="payable",
        amount=110,
        denom="USD",
        asset_holder_id="Agent2",
        liability_issuer_id="Agent4",
        due_day=2
    )
    fresh_sys.add_contract(payment)
    
    delivery2 = DeliveryObligation(
        id=fresh_sys.new_contract_id("DEL2"),
        kind="delivery_obligation",
        amount=10,
        sku="machines",
        unit_price=Decimal("0"),
        denom="USD",
        asset_holder_id="Agent4",
        liability_issuer_id="Agent2",
        due_day=2
    )
    fresh_sys.add_contract(delivery2)

print("✅ Fresh system created with same initial conditions")
print(f"Starting at day: {fresh_sys.state.day}")

# Now run until stable
print("\n🚀 Running simulation until all obligations are settled...")
reports = run_until_stable(fresh_sys, max_days=10, quiet_days=2)

print(f"\n✅ Simulation completed!")
print(f"Final day: {fresh_sys.state.day}")
print(f"Total days simulated: {len(reports)}")

print("\n📊 Day-by-day activity report:")
for report in reports:
    if report.impacted > 0:
        print(f"  Day {report.day}: {report.impacted} impactful events")
    else:
        print(f"  Day {report.day}: No activity (quiet day)")

print("\n🔍 Final state verification:")
balance1 = agent_balance(fresh_sys, "Agent1")
balance2 = agent_balance(fresh_sys, "Agent2")
balance3 = agent_balance(fresh_sys, "Agent3")
balance4 = agent_balance(fresh_sys, "Agent4")

print(f"  Agent1: ${balance1.assets_by_kind.get('cash', 0)} cash")
print(f"  Agent2: ${balance2.total_financial_assets} assets")
print(f"  Agent3: ${balance3.total_financial_assets} assets")
print(f"  Agent4: {balance4.inventory_by_sku.get('machines', {}).get('quantity', 0)} machines")

print("\n✨ The run_until_stable function automatically:")
print("  1. Advanced the simulation day by day")
print("  2. Settled all obligations as they became due")
print("  3. Stopped when no obligations remained and system was quiet")
print("  4. Achieved the exact same final state as manual stepping!")


ALTERNATIVE: RUN UNTIL STABLE DEMONSTRATION

📌 Creating a fresh system to demonstrate automated simulation...
✅ Fresh system created with same initial conditions
Starting at day: 0

🚀 Running simulation until all obligations are settled...

✅ Simulation completed!
Final day: 6
Total days simulated: 6

📊 Day-by-day activity report:
  Day 0: No activity (quiet day)
  Day 1: 1 impactful events
  Day 2: 2 impactful events
  Day 3: 1 impactful events
  Day 4: No activity (quiet day)
  Day 5: No activity (quiet day)

🔍 Final state verification:
  Agent1: $110 cash
  Agent2: $0 assets
  Agent3: $0 assets
  Agent4: 10 machines

✨ The run_until_stable function automatically:
  1. Advanced the simulation day by day
  2. Settled all obligations as they became due
  3. Stopped when no obligations remained and system was quiet
  4. Achieved the exact same final state as manual stepping!


In [11]:
display_events(fresh_sys.state.events)


📅 Day 0:
  🏭 Stock created: Agent3 gets 10 machines
  💰 Cash minted: $110 to Agent4
  ⏰ Settlement phase begins (checking due obligations)
  ⏳ End of day phase
  • PhaseC: {'kind': 'PhaseC', 'day': 0, 'phase': 'simulation'}

📅 Day 1:
  ⏰ Settlement phase begins (checking due obligations)
  ⏳ End of day phase
  📦 Stock transferred: Agent3 → Agent2: 10 machines
    └─ Delivery obligation removed from books
  ✅ Delivery settled: Agent3 → Agent2: 10 machines
  • PhaseC: {'kind': 'PhaseC', 'day': 1, 'phase': 'simulation'}

📅 Day 2:
  ⏰ Settlement phase begins (checking due obligations)
  ⏳ End of day phase
  💵 Cash transferred: Agent4 → Agent2: $110
  ✅ Payment settled: Agent4 → Agent2: $110
  📦 Stock transferred: Agent2 → Agent4: 10 machines
    └─ Delivery obligation removed from books
  ✅ Delivery settled: Agent2 → Agent4: 10 machines
  • PhaseC: {'kind': 'PhaseC', 'day': 2, 'phase': 'simulation'}

📅 Day 3:
  ⏰ Settlement phase begins (checking due obligations)
  ⏳ End of day phase
  💵 